#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 8.9 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [14]:
import pandas as pd

#SC Location Problem

In [3]:
#reading in distance data
import pandas as pd
distdata = pd.read_excel('07_async_sclocationdata.xlsx', sheet_name='distancematrix') 
distdata

,Unnamed: 0,Boston,Chicago,Dallas,Denver,Los Angeles,Miami,New York,Phoenix,Pittsburgh
0,Boston,0,983,1815,1991,3036,1539,213,2664,792
1,Chicago,983,0,1205,1050,2112,1390,840,1729,457
2,Dallas,1815,1205,0,801,1425,1332,1604,1027,1237
3,Denver,1991,1050,801,0,1174,2041,1780,836,1411
4,Los Angeles,3036,2112,1425,1174,0,2757,2825,398,2456
5,Miami,1539,1390,1332,2041,2757,0,1258,2359,1250
6,New York,213,840,1604,1780,2825,1258,0,2442,386
7,Phoenix,2664,1729,1027,836,398,2359,2442,0,2073
8,Pittsburgh,792,457,1237,1411,2456,1250,386,2073,0


In [4]:
#converting distance data to a list of lists
distances = distdata.loc[:, 'Boston':'Pittsburgh'].values.tolist()
distances

[[0, 983, 1815, 1991, 3036, 1539, 213, 2664, 792],
 [983, 0, 1205, 1050, 2112, 1390, 840, 1729, 457],
 [1815, 1205, 0, 801, 1425, 1332, 1604, 1027, 1237],
 [1991, 1050, 801, 0, 1174, 2041, 1780, 836, 1411],
 [3036, 2112, 1425, 1174, 0, 2757, 2825, 398, 2456],
 [1539, 1390, 1332, 2041, 2757, 0, 1258, 2359, 1250],
 [213, 840, 1604, 1780, 2825, 1258, 0, 2442, 386],
 [2664, 1729, 1027, 836, 398, 2359, 2442, 0, 2073],
 [792, 457, 1237, 1411, 2456, 1250, 386, 2073, 0]]

In [5]:
#reading in annual trips data
tripsdata = pd.read_excel('07_async_sclocationdata.xlsx', sheet_name='annualtrips') #Change to your file name and sheet name
tripsdata

,City,Annual trips
0,Boston,885
1,Chicago,760
2,Dallas,1124
3,Denver,708
4,Los Angeles,1224
5,Miami,1152
6,New York,1560
7,Phoenix,1222
8,Pittsburgh,856


In [6]:
#converting annual trips to a list:
annualtrips = tripsdata["Annual trips"].tolist()
annualtrips

[885, 760, 1124, 708, 1224, 1152, 1560, 1222, 856]

In [7]:
#converting names of cities to list:
cities = tripsdata["City"].tolist()
cities

['Boston',
 'Chicago',
 'Dallas',
 'Denver',
 'Los Angeles',
 'Miami',
 'New York',
 'Phoenix',
 'Pittsburgh']

In [18]:

num_svc_locations = 9 
num_ret_locations = 9
number = 10000

#define the optimization model
model = ConcreteModel()

# define decision variables for x and y
model.x = Var(range(num_svc_locations), domain = Binary)
model.y = Var(range(num_ret_locations), range(num_svc_locations), domain = Binary)

# define objective to minimize total distance
tot_dist_trav = sum(model.y[j,i]*distances[j][i]*annualtrips[j] for i in range(num_svc_locations) for j in range(num_ret_locations))
model.Objective = Objective(expr = tot_dist_trav, sense = minimize)

# add in the constraints
model.constraints3 = Constraint(expr= sum(model.x[i] for i in range(num_svc_locations))<= 3)

# city constraint
model.City = ConstraintList()
for j in range(num_ret_locations):
  model.City.add(sum(model.y[j,i] for i in range(num_svc_locations))== 1)

# only if city exists
model.Assign = ConstraintList()
for i in range(num_svc_locations):
  model.Assign.add(sum(model.y[j,i] for j in range(num_ret_locations)) <= model.x[i]*number)

model.pprint()

6 Set Declarations
    Assign_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    City_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}
    y_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : y_index_0*y_index_1 :   81 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 0), (4, 1), (4, 2), (4,

In [19]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpkkfawqpf.glpk.raw --wglp /tmp/tmpuxs23w39.glpk.glp
 --cpxlp /tmp/tmpjszuk0wi.pyomo.lp
Reading problem data from '/tmp/tmpjszuk0wi.pyomo.lp'...
/tmp/tmpjszuk0wi.pyomo.lp:412: warning: lower bound of variable 'x1' redefined
/tmp/tmpjszuk0wi.pyomo.lp:412: warning: upper bound of variable 'x1' redefined
20 rows, 91 columns, 181 non-zeros
90 integer variables, all of which are binary
502 lines were read
Writing problem data to '/tmp/tmpuxs23w39.glpk.glp'...
388 lines were written
GLPK Integer Optimizer, v4.65
20 rows, 91 columns, 181 non-zeros
90 integer variables, all of which are binary
Preprocessing...
9 constraint coefficient(s) were reduced
19 rows, 90 columns, 180 non-zeros
90 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  9.000e+00  ratio =  9.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size 

In [20]:
#print the results
print("Total Distance", model.Objective())
svc_locations = [model.x[i]() for i in range(num_svc_locations)]
city_assignment = [[model.y[j,i]() for i in range(num_svc_locations)] for j in range(num_ret_locations)]

Total Distance 3390709.0
